# Python for Data Analysis v2 | Notes_ Chapter_4 NumPy 基础：数组和矢量计算

本人以简书作者 SeanCheney 系列专题文章并结合原书为学习资源，记录个人笔记，仅作为知识记录及后期复习所用，原作者地址查看 [简书 SeanCheney](https://www.jianshu.com/u/130f76596b02)，如有错误，还望批评指教。——ZJ


>原作者：SeanCheney | [《利用Python进行数据分析·第2版》第4章 NumPy基础：数组和矢量计算](https://www.jianshu.com/p/a380222a3292) | 來源：简书

>[Github:wesm](https://github.com/wesm/pydata-book) | [Github:中文 BrambleXu](https://github.com/BrambleXu/pydata-notebook)|
简书:[利用Python进行数据分析·第2版](https://www.jianshu.com/c/52882df3377a)

环境：Python 3.6 

---

## 第 4 章 NumPy 基础：数组和矢量计算


NumPy（Numerical Python 的简称）是 Python数 值计算最重要的基础包。大多数提供科学计算的包都是用 NumPy 的数组作为构建基础。

NumPy 的部分功能如下：

- ndarray，一个具有矢量算术运算和复杂广播能力的快速且节省空间的多维数组。
- 用于对整组数据进行快速运算的标准数学函数（无需编写循环）。
- 用于读写磁盘数据的工具以及用于操作内存映射文件的工具。
- 线性代数、随机数生成以及傅里叶变换功能。
- 用于集成由 C、C++、Fortran 等语言编写的代码的 A C API。


对于大部分数据分析应用而言，我最关注的功能主要集中在：

- 用于数据整理和清理、子集构造和过滤、转换等快速的矢量化数组运算。
- 常用的数组算法，如排序、唯一化、集合运算等。
- 高效的描述统计和数据聚合/摘要运算。
- 用于异构数据集的合并/连接运算的数据对齐和关系型数据运算。
- 将条件逻辑表述为数组表达式（而不是带有if-elif-else分支的循环）。
- 数据的分组运算（聚合、转换、函数应用等）

NumPy 之于数值计算特别重要的原因之一，是因为它可以高效处理大数组的数据。这是因为：

- NumPy 是在一个连续的内存块中存储数据，独立于其他 Python 内置对象。NumPy 的 C 语言编写的算法库可以操作内存，而不必进行类型检查或其它前期工作。比起 Python 的内置序列，NumPy 数组使用的内存更少。
- NumPy 可以在整个数组上执行复杂的计算，而不需要 Python 的 for 循环。

要搞明白具体的性能差距，考察一个包含一百万整数的数组，和一个等价的Python 列表:

```
In [1]: import numpy as np

In [2]: my_arr = np.arange(1000000)

In [3]: my_list = list(range(1000000))

# 各个序列分别乘以 2：

In [4]: %time for _ in range(10):my_arr2 = my_arr * 2
Wall time: 46.9 ms

In [5]: %time for _ in range(10): my_list = [x * 2 for x in my_list]
Wall time: 913 ms


```

基于 NumPy 的算法要比纯 Python 快 10 到 100 倍（甚至更快），并且使用的内存更少。

### 4.1 NumPy 的 ndarray：一种多维数组对象

NumPy 最重要的一个特点就是其 N 维数组对象（即 ndarray），该对象是一个快速而灵活的大数据集容器。

Python 是如何利用与标量值类似的语法进行批次计算:

```
In [6]: data = np.random.randn(2,3)

In [7]: data
Out[7]:
array([[-0.7949888 ,  0.59269599,  0.6047145 ],
       [-0.05579778, -1.97623901, -0.52438662]])

In [8]: data * 10
Out[8]:
array([[ -7.94988799,   5.92695992,   6.04714496],
       [ -0.55797776, -19.76239014,  -5.24386616]])

In [9]: data + data
Out[9]:
array([[-1.5899776 ,  1.18539198,  1.20942899],
       [-0.11159555, -3.95247803, -1.04877323]])


```

ndarray 是一个通用的同构数据多维容器，也就是说，其中的所有元素必须是相同类型的。每个数组都有一个 shape（一个表示各维度大小的元组）和一个dtype（一个用于说明数组数据类型的对象）：


```
In [10]: data.shape
Out[10]: (2, 3)

In [11]: data.dtype
Out[11]: dtype('float64')


```

- 笔记：当你在本书中看到“数组”、“NumPy 数组”、"ndarray"时，基本上都指的是同一样东西，即 ndarray 对象。


####  创建 ndarray

创建数组最简单的办法就是使用 array 函数。它接受一切序列型的对象（包括其他数组），然后产生一个新的含有传入数据的 NumPy 数组。以一个列表的转换为例：

```
In [12]: data1 = [6, 7.5, 8, 0, 1]

In [13]: arr1 = np.array(data1)

In [14]: arr1
Out[14]: array([6. , 7.5, 8. , 0. , 1. ])

# 嵌套序列（比如由一组等长列表组成的列表）将会被转换为一个多维数组：

In [15]: data2 = [[1,2,3,4],[5,6,7,8]]

In [17]: arr2 = np.array(data2)

In [18]: arr2
Out[18]:
array([[1, 2, 3, 4],
       [5, 6, 7, 8]])


```
因为 data2 是列表的列表，NumPy 数组 arr2 的两个维度的 shape 是从 data2引入的。可以用属性 ndim 和 shape 验证：

```
In [19]: arr2.ndim
Out[19]: 2

In [20]: arr2.shape
Out[20]: (2, 4)


```

除非特别说明（稍后将会详细介绍），np.array 会尝试为新建的这个数组推断出一个较为合适的数据类型。数据类型保存在一个特殊的dtype对象中。比如说，在上面的两个例子中，我们有：

```
In [21]: arr1.dtype
Out[21]: dtype('float64')

In [22]: arr2.dtype
Out[22]: dtype('int32')


```

除 np.array 之外，还有一些函数也可以新建数组。比如，zeros 和 ones 分别可以创建指定长度或形状的全 0 或全 1 数组。empty 可以创建一个没有任何具体值的数组。要用这些方法创建多维数组，只需传入一个表示形状的元组即可：


```
In [26]: np.zeros(10)
Out[26]: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [27]: np.zeros((4,5))
Out[27]:
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [28]: np.empty((2,3,2))
Out[28]:
array([[[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]]])


```
注意：认为`np.empty`会返回全 0 数组的想法是不安全的。很多情况下（如前所示），它返回的都是一些未初始化的垃圾值。

- arange 是 Python 内置函数 range 的数组版：

```
In [29]: np.arange(12)
Out[29]: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

```

表 4-1 列出了一些数组创建函数。由于 NumPy 关注的是数值计算，因此，如果没有特别指定，数据类型基本都是 float64（浮点数）。

![表4-1 数组创建函数](./images/numpy.png)


#### ndarray 的数据类型

- dtype（数据类型）是一个特殊的对象，它含有 ndarray 将一块内存解释为特定数据类型所需的信息：

```
In [31]: arr1 = np.array([1,2,3], dtype=np.float64)

In [32]: arr2 = np.array([1,2,3], dtype=np.int32)

In [33]: arr1
Out[33]: array([1., 2., 3.])

In [34]: arr2
Out[34]: array([1, 2, 3])

In [35]: arr1.dtype
Out[35]: dtype('float64')

In [36]: arr2.dtype
Out[36]: dtype('int32')


```

- dtype 是 NumPy 灵活交互其它系统的源泉之一。多数情况下，它们直接映射到相应的机器表示，这使得“读写磁盘上的二进制数据流”以及“集成低级语言代码（如 C、Fortran）”等工作变得更加简单。数值型 dtype 的命名方式相同：一个类型名（如 float 或 int），后面跟一个用于表示各元素位长的数字。标准的双精度浮点值（即 Python 中的 float对象）需要占用 8 字节（即64 位）。因此，该类型在 NumPy 中就记作 float64。表 4-2 列出了 NumPy 所支持的全部数据类型。

- 笔记：记不住这些 NumPy 的 dtype 也没关系，新手更是如此。通常只需要知道你所处理的数据的大致类型是浮点数、复数、整数、布尔值、字符串，还是普通的 Python 对象即可。当你需要控制数据在内存和磁盘中的存储方式时（尤其是对大数据集），那就得了解如何控制存储类型。


![](./images/data_type1.png)
![](./images/data_type2.png)


- 你可以通过 ndarray 的 astype 方法明确地将一个数组从一个 dtype 转换成另一个 dtype：

```
In [37]: arr = np.array([1,2,3,4,5])

In [38]: arr.dtype
Out[38]: dtype('int32')

In [39]: float_arr = arr.astype(np.float64)

In [40]: float_arr.dtype
Out[40]: dtype('float64')


```

- 在本例中，整数被转换成了浮点数。如果将浮点数转换成整数，则小数部分将会被截取删除：

```
In [41]: arr = np.array([3.7, -1.2, -2.6, 0.5, 12.9, 10.1])

In [42]: arr
Out[42]: array([ 3.7, -1.2, -2.6,  0.5, 12.9, 10.1])

In [43]: arr.astype(np.int32)
Out[43]: array([ 3, -1, -2,  0, 12, 10])

```

- 如果某字符串数组表示的全是数字，也可以用 astype 将其转换为数值形式：

```
In [44]: numeric_strings = np.array(['1.25', '9.6', '45'], dtype=np.s
    ...: tring_ )

In [45]: numeric_strings.astype(float)
Out[45]: array([ 1.25,  9.6 , 45.  ])

```

- 注意：使用`numpy.string_`类型时，一定要小心，因为 NumPy 的字符串数据是大小固定的，发生截取时，不会发出警告。pandas 提供了更多非数值数据的便利的处理方法。

- 数组的 dtype 还有另一个属性：

```
In [46]: int_array = np.arange(10)

In [47]: calibers = np.array([.22, .270, .357, .380, .44, .50], dtype=np.float64)

In [48]: int_array.astype(calibers.dtype)
Out[48]: array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

# 你还可以用简洁的类型代码来表示 dtype：

In [49]: empty_uint32 = np.empty(8, dtype='u4')

In [50]: empty_uint32
Out[50]: 
array([         0, 1075314688,          0, 1075707904,          0,
       1075838976,          0, 1072693248], dtype=uint32)

```
- 笔记：调用 astype 总会创建一个新的数组（一个数据的备份），即使新的dtype 与旧的 dtype 相同。

#### NumPy 数组的运算 

#### 基本的索引和切片

#### 切片索引

#### 布尔型索引

#### 花式索引

#### 数组转置和轴对换



### 4.2 通用函数：快速的元素级数组函数

####

### 4.3 利用数组进行数据处理

#### 将条件逻辑表述为数组运算

#### 数学和统计方法

#### 用于布尔型数组的方法

#### 排序

#### 唯一化以及其它的集合逻辑

### 4.4 用于数组的文件输入输出

### 4.5 线性代数 


### 4.6 伪随机数生成

### 4.7 示例：随机漫步

#### 一次模拟多个随机漫步

###  4.8 结论

虽然本书剩下的章节大部分是用 pandas 规整数据，我们还是会用到相似的基于数组的计算。在附录A中，我们会深入挖掘 NumPy 的特点，进一步学习数组的技巧。